In [1]:
using SymPy
using LinearAlgebra

## 旋量度规的确定

【确定依据】：**旋量度规在自旋变换的作用下不变**

#### 1）自旋矩阵的约束

自旋矩阵$L^\Sigma_{\ \ \Omega}\in \mathrm{SU}(2)$:

$$L^\dagger L=I,\quad \det L=1$$


In [2]:
@vars a b c d

L=[a b; c d]

2×2 Array{Sym,2}:
 a  b
 c  d

约束条件1： $ \det L=1$

In [3]:
cond1 = solve(det(L)-1,[a*d]);
cond1 = a*d=>cond1[1]

a*d => b*c + 1

约束条件2：$L^\dagger L=I$

In [4]:
# L的逆矩阵（带入条件1）
Li= L |> inv .|> (expr->expr(cond1));

# L的共轭转置
Lc = L.T  .|> conj;

eqs1 = Li-Lc;
eqs1 = [eqs1[i] for i in 1:4];
cond2 = solve(eqs1,[c,d])[1];
cond2 = c=>cond2[1],d=>cond2[2]

(c => -conjugate(b), d => conjugate(a))

####  2）旋量度规在自旋变换的作用下不变

$$\epsilon_{AB} L^A_{\ \ C}L^B_{\ \ D}= \epsilon_{CD}$$

In [5]:
@vars α β γ ζ

# 待定变量
ϵ=[α β; γ ζ];


# 列方程
eqs=[sum(ϵ[A,B]*L[A,C]*L[B,D] for A in 1:2, B in 1:2)-ϵ[C,D] for C in 1:2, D in 1:2];
eqs = eqs .|> (expr->expr(cond1)) .|> expand .|> (expr->expr(cond2[1],cond2[2])) .|> simplify;
eqs = [eqs[i] for i in 1:4]

4-element Array{Sym,1}:
        a^2*α - a*β*conjugate(b) - a*γ*conjugate(b) - α + ζ*conjugate(b)^2
 a*b*α - b*β*conjugate(b) - b*γ*conjugate(b) - ζ*conjugate(a)*conjugate(b)
 a*b*α - b*β*conjugate(b) - b*γ*conjugate(b) - ζ*conjugate(a)*conjugate(b)
        b^2*α + b*β*conjugate(a) + b*γ*conjugate(a) + ζ*conjugate(a)^2 - ζ

In [6]:
# 解方程
solve(eqs,[α,β,γ,ζ])

Dict{Any,Any} with 3 entries:
  α => 0
  β => -γ
  ζ => 0

## 时空度规和旋量度规

$$g_{\mu\upsilon}=\mathrm{det}(-1,1,1,1)\\ g^{\mu\upsilon}=\mathrm{det}(-1,1,1,1)\\ \delta^\mu_\upsilon=\det (1,1,1,1)$$

$$\epsilon_{\Sigma\Omega}=\epsilon^{\Sigma\Omega}=\begin{pmatrix} 0 & 1 \\  -1& 0 \end{pmatrix}\\ \bar{\epsilon}_{\Sigma'\Omega'}=\bar{\epsilon}^{\Sigma'\Omega'}=\begin{pmatrix} 0 & 1 \\  -1 & 0 \end{pmatrix}$$

In [7]:
δ = sympy.eye(4); 

g = sympy.diag(-1, 1, 1, 1);
gi = inv(g);

ϵ = sympy.Matrix([0 1; -1 0]);

## 变换的行列式

$$\det(L)=\frac{1}{2}\epsilon_{AB}\epsilon^{CD}L^A_{\ \ C}L^B_{\ \ D}$$

In [8]:
[sum(ϵ[A,B]*ϵ[C,D]*L[A,C]*L[B,D]//2 for A in 1:2, B in 1:2, C in 1:2, D in 1:2)]

1-element Array{Sym,1}:
 a*d - b*c

## 旋量度规的性质


#### 1）性质1：

$$\epsilon^A_{\ \ C}=\epsilon^{AB}\epsilon_{BC}=-\delta^A_C =\omicron^A\iota_C-\iota^A\omicron_C\\ \epsilon^\Sigma_{\ \ \Omega}=\begin{pmatrix} 0 & 1 \\ -1 & 0 \end{pmatrix}$$


In [9]:
[sum(ϵ[A,B]*ϵ[B,C] for B in 1:2) for A in 1:2, C in 1:2] # |> println

2×2 Array{Sym,2}:
 -1   0
  0  -1

#### 2) 性质2：

$$\epsilon_{AB}=\epsilon^{CD}\epsilon_{CA}\epsilon_{DB}$$

$$\bar{\epsilon}_{A'B'}=\bar{\epsilon}^{C'D'}\bar{\epsilon}_{C'A'}\bar{\epsilon}_{D'B'}$$

$$\bar{\epsilon}^{A'B'}\bar{\epsilon}_{B'C'}=-\delta^A_C$$


In [10]:
[sum(ϵ[C,D]*ϵ[C,A]*ϵ[D,B] for C in 1:2, D in 1:2) for A in 1:2, B in 1:2]

2×2 Array{Sym,2}:
  0  1
 -1  0

In [11]:
[sum(ϵ[C1,D1]ϵ[C1,A1]ϵ[D1,B1] for C1 in 1:2, D1 in 1:2) for A1 in 1:2, B1 in 1:2]

2×2 Array{Sym,2}:
  0  1
 -1  0

In [12]:
[sum(ϵ[A1,B1]*ϵ[B1,C1] for B1 in 1:2) for A1 in 1:2, C1 in 1:2]

2×2 Array{Sym,2}:
 -1   0
  0  -1

## 泡利矩阵及其连带矩阵

$$\sigma_\mu^{\ \ AA'}$$

In [13]:
σ_0 = sympy.Matrix([1 0;0 1])*sqrt(1//2);
σ_1 = sympy.Matrix([0 1;1 0])*sqrt(1//2);
σ_2 = sympy.Matrix([0 -IM;IM 0])*sqrt(1//2);
σ_3 = sympy.Matrix([1 0;0 -1])*sqrt(1/2);
σ=[σ_0,σ_1,σ_2,σ_3];
σ=[σ[i][j,k] for i in 1:4,j in 1:2 ,k in 1:2];
[σ[i,j,k] for j in 1:2 ,k in 1:2,  i in 1:4]

2×2×4 Array{Sym,3}:
[:, :, 1] =
 0.707106781186548                  0
                 0  0.707106781186548

[:, :, 2] =
                 0  0.707106781186548
 0.707106781186548                  0

[:, :, 3] =
                   0  -0.707106781186548*I
 0.707106781186548*I                     0

[:, :, 4] =
 0.707106781186548                   0
                 0  -0.707106781186548

$$\sigma^\mu_{\ \ AA'}=g^{\mu\upsilon}\sigma_\upsilon^{\ \ BB'}\epsilon_{BA}\bar{\epsilon}_{B'A'}$$

In [14]:
σi=[sum(gi[μ,υ]*σ[υ,B,B1]*ϵ[B,A]*ϵ[B1,A1] for υ in 1:4, B in 1:2, B1 in 1:2) 
    for μ in 1:4, A in 1:2 , A1 in 1:2];

[sum(gi[μ,υ]*σi[υ,A,B] for υ in 1:4) for A in 1:2 ,B in 1:2, μ in 1:4]

2×2×4 Array{Sym,3}:
[:, :, 1] =
 0.707106781186548                  0
                 0  0.707106781186548

[:, :, 2] =
                  0  -0.707106781186548
 -0.707106781186548                   0

[:, :, 3] =
                   0  -0.707106781186548*I
 0.707106781186548*I                     0

[:, :, 4] =
 -0.707106781186548                  0
                  0  0.707106781186548

$$\sigma_\mu^{\ \ AA'}\sigma^\upsilon_{\ \ AA'}=-\delta^\upsilon_\mu\\ \sigma_\mu^{\ \ AA'}\sigma^\mu_{\ \ BB'}=-\delta^A_B\delta^{A'}_{B'}$$

In [15]:
[sum(σ[μ,A,A1]*σi[υ,A,A1] for A in 1:2, A1 in 1:2) for υ in 1:4, μ in 1:4]

4×4 Array{Sym,2}:
 -1.00000000000000                  0                  0                  0
                 0  -1.00000000000000                  0                  0
                 0                  0  -1.00000000000000                  0
                 0                  0                  0  -1.00000000000000

In [16]:
[sum(σ[μ,A,A1]*σi[μ,B,B1] for μ in 1:4)+δ[A,B]*δ[A1,B1] for A in 1:2, A1 in 1:2, B in 1:2, B1 in 1:2 ]

2×2×2×2 Array{Sym,4}:
[:, :, 1, 1] =
 -2.22044604925031e-16  0
                     0  0

[:, :, 2, 1] =
                     0  0
 -2.22044604925031e-16  0

[:, :, 1, 2] =
 0  -2.22044604925031e-16
 0                      0

[:, :, 2, 2] =
 0                      0
 0  -2.22044604925031e-16

$g^{\mu\upsilon}\sigma_\mu^{\ \ AA'}\sigma_\upsilon^{BB'}-\epsilon^{AB}\epsilon^{\ \ A'B'}=0$

这个直接验证很麻烦

In [17]:
[sum(gi[μ,υ]*σ[μ,A,A1]*σ[υ,B,B1] for μ in 1:4, υ in 1:4)+ϵ[A,B]*ϵ[A1,B1]
        for A in 1:2, B in 1:2, A1 in 1:2, B1 in 1:2]

2×2×2×2 Array{Sym,4}:
[:, :, 1, 1] =
 0  0
 0  0

[:, :, 2, 1] =
                     0  2.22044604925031e-16
 -2.22044604925031e-16                     0

[:, :, 1, 2] =
                    0  -2.22044604925031e-16
 2.22044604925031e-16                      0

[:, :, 2, 2] =
 0  0
 0  0

$\sigma_\mu^{\ \ AA'}\sigma^\upsilon_{\ \ AA'}=\delta^\upsilon_\mu$

In [18]:
[sum(sum(σ[μ,A,A1]*σi[υ,A,A1] for A in 1:2) for A1 in 1:2) for μ in 1:4, υ in 1:4]

4×4 Array{Sym,2}:
 -1.00000000000000                  0                  0                  0
                 0  -1.00000000000000                  0                  0
                 0                  0  -1.00000000000000                  0
                 0                  0                  0  -1.00000000000000

$\sigma_\mu^{\ \ AA'}\sigma^\mu_{\ \ BB'}-\delta^A_B\delta^{A'}_{B'}$

In [19]:
[sum(σ[μ,A,A1]*σi[μ,B,B1] for μ in 1:4)-δ[A,B]*δ[A1,B1] for A in 1:2, B in 1:2, A1 in 1:2, B1 in 1:2]

2×2×2×2 Array{Sym,4}:
[:, :, 1, 1] =
 -2.00000000000000                  0
                 0  -2.00000000000000

[:, :, 2, 1] =
 0  0
 0  0

[:, :, 1, 2] =
 0  0
 0  0

[:, :, 2, 2] =
 -2.00000000000000                  0
                 0  -2.00000000000000